In [ ]:
import datetime
import os.path
import sys

import pandas as pd
import numpy as np

import backtrader as bt

class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('maperiod2', 200),
        ('printlog', False),
    )
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt= dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')
            
    def __init__(self):
        self.dataclose = self.datas[0].close
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0],  period= self.params.maperiod
        )
        self.sma2 = bt.indicators.SimpleMovingAverage(
            self.datas[0],  period= self.params.maperiod2
        )
            
        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order= None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')
            
    def next(self):
        self.log(f'Close, {self.dataclose[0]:.2f}')
        
        if self.order:
            return
        
        if not self.position:
            if  self.sma2[0] > self.sma[0]:  
                self.log(f'BUY CREATE, {self.dataclose[0]:.2f}')
                self.order = self.buy()
        else:
            if  self.sma2[0] < self.sma[0]:
                self.log(f"SELL CREATE {self.dataclose[0]:.2f}")
                self.order = self.sell()
            
    def stop(self):
        self.log(f'''(MA Period {self.params.maperiod:2d}) \
                 (MA Period2 {self.params.maperiod2:2d}) \
                Ending Value {self.broker.getvalue():.2f}''', doprint=True)

In [ ]:
class myAnalyzer(bt.Analyzer):

    def start(self):
        self.trade_count = 0
        self.starting_nav= self.strategy.broker.get_value() 

    def notify_trade(self, trade):
        self.trade_count += 1
    
    def stop(self):
        self.ending_nav = self.strategy.broker.getvalue()
        self.params= self.strategy.params

    def get_analysis(self):
        ret= {  'trade_count': self.trade_count,
                'ending_NAV': self.ending_nav,
                'P&L' : self.ending_nav-self.starting_nav }
        ret.update(**vars(self.params))
        return ret

In [ ]:
cerebro = bt.Cerebro()

# Add Strategy
opt_args=dict(
    maperiod=[2,3,5,8,13,21,34,55,89,144,500],
    maperiod2=[5,8,13,21,34,55,89,144,233,500,1000]
)

cerebro.optstrategy(TestStrategy, **opt_args)

# Add analyzers

cerebro.addanalyzer(myAnalyzer)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Transactions)

# Load data

def get_data(fname_symbol = 'CL', folder_name = '5min', suffix = '5min_20160103_20190405'):
    df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
    df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
    df.columns = [col_name.lower() for col_name in df.columns]
    df = df.dropna()
    return df


df=get_data()
data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(data,name="CL")

# Set broker parameters

cerebro.broker.setcash(100000.)
cerebro.broker.setcommission(commission=2.5, margin=6000.0, mult=1000.0, name='CL')
cerebro.broker.set_slippage_fixed(.01, 
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=False, 
                              slip_out=True)

# Set sizing

cerebro.addsizer(bt.sizers.FixedSize, stake=1)


# Go

results=cerebro.run()


In [ ]:
analysis = pd.DataFrame(m.analyzers.myanalyzer.get_analysis() for [m] in results)'printlog
analysis = analysis.set_index(['maperiod', 'maperiod2']).sort_values('ending_NAV',ascending=False).drop(', axis=1)

threshold=len(opt_args)*50
qualified = analysis[analysis.trade_count>threshold]
        
best_parameters=dict(zip(qualified.index.names,qualified.index[0]))

## Insample for best performing parameters:

In [ ]:
import sys;sys.path.append("..")
from extra import basictradestats #from https://community.backtrader.com/topic/670/it-s-here-a-beta-you-can-use-right-now-essential-trade-statistics-all-in-one-place/2
# also http://actuarialdatascience.com/backtrader_performance_report.html

%matplotlib inline
cerebro=bt.Cerebro()
data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(data, name="CL")
cerebro.addstrategy(TestStrategy, **best_parameters,printlog=False)

# Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, riskfreerate=0,_name='mysharpe',timeframe=bt.TimeFrame.Days)#, compression=30)
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days, _name='dayreturn')
cerebro.addanalyzer(basictradestats.BasicTradeStats)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)
cerebro.broker.setcommission(commission=2.5, margin=6000.0, mult=1000.0, name='CL')
cerebro.broker.set_slippage_fixed(.01, 
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=False, 
                              slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result=cerebro.run()

print('Sharpe Ratio:', result[0].analyzers.mysharpe.get_analysis()['sharperatio'])

daily_returns = result[0].analyzers.dayreturn.get_analysis()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(iplot=True)

result[0].analyzers.basictradestats.print()

## Out-of-sample for best performing parameters:

In [ ]:
%matplotlib inline
cerebro=bt.Cerebro()
data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])
cerebro.adddata(data, name="CL")
cerebro.addstrategy(TestStrategy, **best_parameters,printlog=False)

# Analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, riskfreerate=0,_name='mysharpe',timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days, _name='dayreturn')
cerebro.addanalyzer(basictradestats.BasicTradeStats)


# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)
cerebro.broker.setcommission(commission=2.5, margin=6000.0, mult=1000.0, name='CL')
cerebro.broker.set_slippage_fixed(.01, 
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=False, 
                              slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result=cerebro.run()

print('Sharpe Ratio:', result[0].analyzers.mysharpe.get_analysis()['sharperatio'])

daily_returns = result[0].analyzers.dayreturn.get_analysis()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(iplot=True)

result[0].analyzers.basictradestats.print()